In [101]:
import math
def AMS(s, b):
    return round(math.sqrt(2*((s+b)*math.log(1+s/b)-s)),3)
def percentage(a,b): return round(abs((a-b)*100/(a+b)),3)

In [91]:
!ls ../results/2018/makeSimple_DF

output_test_data.coffea     output_test_ewk.coffea     output_test_st.coffea
output_test_diboson.coffea  output_test_higgs.coffea   output_test_tt.coffea
output_test_dy.coffea	    output_test_signal.coffea  output_test_wjets.coffea


In [115]:
from coffea.util import load
import numpy as np
import pandas as pd
import glob, os, json, argparse
years = ['2016preVFP', '2017', '2016postVFP','2018']
var_dict = [{}, {}, {}]
df = []
for year in years:
    print(f'Processing {year}')
    for subsample in ['signal','diboson','tt','st','dy','wjets','higgs','ewk']: #'data',
        result = load(f"../results/{year}/makeSimple_DF/output_test_{subsample}.coffea")
        if isinstance(result,tuple):
            result = result[0]
        for varName in result:
            for i in range(3):
                if f'{i}jets' in varName:
                    if varName.replace(f'_{i}jets','') in var_dict[i]:
                        var_dict[i][varName.replace(f'_{i}jets','')] = np.append(var_dict[i][varName.replace(f'_{i}jets','')],result[varName].value)
                    else:
                        var_dict[i][varName.replace(f'_{i}jets','')] = result[varName].value
for i in range(3):
    df.append(pd.DataFrame(var_dict[i]))
data_cuts_os = pd.concat([df[i] for i in range(3)], axis=0, ignore_index=True)

Processing 2016preVFP
Processing 2017
Processing 2016postVFP
Processing 2018


In [116]:
samples = ['data', 'GluGlu_LFV_HToEMu_M125', 'VBF_LFV_HToEMu_M125', 'TTTo2L2Nu', 'ZZ_TuneCP5', 'WZ_TuneCP5', 'WW_TuneCP5', 'DYJetsToLL_M-50', 'DYJetsToLL_M-10to50', 'WGToLNuG_TuneCP5', 'TTToSemiLeptonic', 'TTToHadronic', 'ST_tW_antitop_5f_inclusiveDecays', 'ST_tW_top_5f_inclusiveDecays', 'ST_t-channel_antitop_4f_InclusiveDecays', 'ST_t-channel_top_4f_InclusiveDecays', 'EWKZ2Jets_ZToLL', 'EWKZ2Jets_ZToNuNu', 'EWKWMinus2Jets', 'EWKWPlus2Jets', 'GluGluHToTauTau', 'VBFHToTauTau', 'WminusHToTauTau', 'WplusHToTauTau', 'GluGluHToWWTo2L2Nu', 'VBFHToWWTo2L2Nu', 'ZHToTauTau', 'WJetsToLNu_TuneCP5']
[samples.index(i) for i in samples if 'TT' in i]
data_cuts_os.loc[((data_cuts_os['sample']==3) | (data_cuts_os['sample']==10) | (data_cuts_os['sample']==11)), ['weight']] *= 1.0711607711238926

In [117]:
data_m = data_cuts_os[data_cuts_os.mtrigger.eq(1)]
for pt in range(20,31):
    bkg = data_m[data_m.ept.gt(pt) & data_m.label.eq(0)]['weight'].sum()
    sig = data_m[data_m.ept.gt(pt) & data_m.label.eq(1)]['weight'].sum()
    print(pt, AMS(sig*0.0059, bkg), round(sig*0.0059), round(bkg))

20 0.959 138 20715
21 0.96 138 20581
22 0.961 138 20429
23 0.962 137 20266
24 0.963 137 20101
25 0.964 136 19931
26 0.964 136 19749
27 0.965 135 19557
28 0.965 134 19350
29 0.965 134 19129
30 0.966 133 18884


In [107]:
data_em = data_cuts_os[(data_cuts_os.etrigger.eq(1) & data_cuts_os.mpt.gt(15))|(data_cuts_os.mtrigger.eq(1) & data_cuts_os.ept.gt(24))]
data_m = data_cuts_os[data_cuts_os.mtrigger.eq(1) & data_cuts_os.ept.gt(24)]
totalbkg_m, totalbkg_em = data_m[data_m.label.eq(0)]['weight'].sum(), data_em[data_em.label.eq(0)]['weight'].sum()
totalsig_m, totalsig_em = data_m[data_m.label.eq(1)]['weight'].sum(), data_em[data_em.label.eq(1)]['weight'].sum()
print('bkg', totalbkg_m, totalbkg_em, percentage(totalbkg_m, totalbkg_em))
print('sig', totalsig_m*0.0059, totalsig_em*0.0059, percentage(totalsig_m, totalsig_em))
print('m', AMS(totalsig_m*0.0059, totalbkg_m))
print('em', AMS(totalsig_em*0.0059, totalbkg_em))
print(percentage(AMS(totalsig_m*0.0059, totalbkg_m), AMS(totalsig_em*0.0059, totalbkg_em)))

bkg 20100.653502799418 23454.67450137778 7.701
sig 136.68773354052857 149.74823727980936 4.56
m 0.963
em 0.977
0.722
